# References

GDS allows defining the component once in memory and reference to that structure in other components.


In [ ]:
from IPython.display import Image

In [ ]:
Image("../docs/images/gds.png")

## Adding a component reference

As we build complex circuit we'll make circuits that combine reference to other simpler circuits. Adding a reference is like having a pointer to the other devices.

In [ ]:
import pp

We have two ways to add a reference to our device:

1. create the reference and add it to the component

In [ ]:
c = pp.Component()
w = pp.c.waveguide(width=0.6)
wr = w.ref()
c.add(wr)
pp.qp(c)

In [ ]:
wr.

2. or we can do it in a single line (my preference)

In [ ]:
c = pp.Component()
wr = c << pp.c.waveguide(width=0.6)
pp.qp(c)

in both cases we can move the reference `wr` after created

In [ ]:
c = pp.Component()
wr1 = c << pp.c.waveguide(width=0.6)
wr2 = c << pp.c.waveguide(width=0.6)
wr2.movey(10)
pp.qp(c)

# Adding a reference array

We can also add an array of references for periodic structures. Lets create a [Distributed Bragg Reflector](https://picwriter.readthedocs.io/en/latest/components/dbr.html)


As PHIDL tutorial says, in GDS, there's a type of structure called a "CellArray" which takes a cell and repeats it NxM times on a fixed grid spacing. For convenience, PHIDL includes this functionality with the add_array() function.  Note that CellArrays are not compatible with ports (since there is no way to access/modify individual elements in a GDS cellarray) 

In [ ]:
@pp.autoname
def dbr_cell(w1=0.5, w2=0.6, l1=0.2, l2=0.4, waveguide_function=pp.c.waveguide):
    c = pp.Component()
    c1 = c << waveguide_function(length=l1, width=w1)
    c2 = c << waveguide_function(length=l2, width=w2)
    c2.connect(port="W0", destination=c1.ports["E0"])
    c.add_port('W0', port=c1.ports['W0'])
    c.add_port('E0', port=c2.ports['E0'])
    return c

w1=0.5
w2=0.6
l1=0.2
l2=0.4
n=3
waveguide_function=pp.c.waveguide
c = pp.Component()
cell = dbr_cell(w1=w1, w2=w2, l1=l1, l2=l2, waveguide_function=waveguide_function)
pp.qp(cell)

In [ ]:
cell_array = c.add_array(device=cell, columns=n, rows=1, spacing=(l1+l2, 100))

In [ ]:
pp.qp(c)

Finally we need to add ports to the new component

In [ ]:
p0 = c.add_port('W0', port=cell.ports['W0'])
p1 = c.add_port('E0', port=cell.ports['E0'])
p1.midpoint = [(l1+l2)*n, 0]

In [ ]:
pp.qp(c)